In [19]:
import rasterio
import pandas as pd
import numpy as np
from pyproj import Transformer
from dbfunctions.heavydb import HeavyDB
from configuration import Configuration

In [20]:
def creat_connection():
    conf = Configuration('heavy.env')
    heavy = HeavyDB()
    heavy.connect(conf)
    heavy.conn.execute('SHOW TABLES')
    return heavy
    
def load_table(df, tablename, schema = 'Band Double, Lon Double, Lat Double'):
    heavy = creat_connection()
    heavy.conn.execute(f'Create Table if not exists {tablename}( {schema});')
    heavy.conn.load_table_columnar(tablename, df)
    return    
def load_data2(tablename, where = ''):
    heavy = creat_connection()
    df = pd.DataFrame(heavy.conn.execute(f"select *,site_id||'_'||cell_id  as cell_index from {tablename} {where};"))
    return df 
def load_data(tablename, where = ''):
    heavy = creat_connection()
    df = pd.DataFrame(heavy.conn.execute(f"select * from {tablename} {where};"))
    return df

In [21]:
import tqdm 
def execute_rfp(i,heavy):
        
        sites = """  full_aoi_all_site_data_suff_Macro where frequency = 1800 """ 
        clutter = f'full_aoi_dsm_utm_area_{i}'
        tqdm.tqdm.write(clutter)
        query = f""" select * from TABLE(tf_rf_prop_max_signal(
            rf_sources => CURSOR(
              SELECT
                  CAST(cell_index as VARCHAR),
                  CAST(x_4326 as FLOAT),
                  CAST(y_4326 as FLOAT),
                  CAST(height as FLOAT) as adjusted_height,
                  CAST((power(10,pow/100)*0.0000004) as FLOAT),
                  CAST(frequency as FLOAT),
                  CAST(azimuth as FLOAT),
                  CAST(mdt + edt AS FLOAT) as downtilt,
                  CAST('V2_Sinclair_SP42F2X_HF1P65LDF(D00)_831_ant' as VARCHAR)
              FROM
                  {sites}
            ),
            terrain_elevations => CURSOR(
              SELECT
                  CAST(x_4326 as DOUBLE),
                  CAST(y_4326 as DOUBLE),
                  CAST(dtm as DOUBLE),
                  CAST(dsm as DOUBLE),
                  CAST(attenuation as DOUBLE) as terrain_attenuation_dbm_per_meter
              FROM
                  {clutter}
            ),
            antenna_patterns => CURSOR(
                SELECT
                    CAST(antenna_name as VARCHAR),
                    CAST(gain_str as DOUBLE),
                    h_pattern_keys,
                    h_pattern_values,
                    v_pattern_keys,
                    v_pattern_values
                FROM
                  antenna_data
            ),
            rf_source_z_is_relative_to_terrain => true,
            geographic_coords => true,
            bin_dim_meters => 5.0,
            assumed_receiver_height_agl => 2.0 ,
            max_ray_travel_meters => 5000.0 ,
            initial_rays_per_source => 64 ,
            min_receiver_signal_strength_dbm => -140,
            default_source_height_agl_meters =>  30.0,
            ray_step_bin_multiple => 1.0,
            loop_grain_size => 8,
            num_rays_per_source => 1024)) where max_rf_signal_strength_dbm is not null """
         
        
        rf_output = pd.DataFrame(heavy.conn.execute(query))
        rf_output.columns = ['x','y','elevation_amsl_meters','rf_source_id','max_rf_signal_strength_dbm']
        return rf_output


def execute_rfp_detailed(i,cell,heavy):
    
    
    sites = f" full_aoi_all_site_data_suff_Macro where cell_index = '{cell}'"
    clutter = f'full_aoi_dsm_utm_area_{i}'
    query = f""" select * from TABLE(tf_rf_prop_max_signal(
        rf_sources => CURSOR(
          SELECT
              CAST(cell_index as VARCHAR),
              CAST(x_4326 as FLOAT),
              CAST(y_4326 as FLOAT),
              CAST(height as FLOAT) as adjusted_height,
              CAST((power(10,pow/100)*0.0000004) as FLOAT),
              CAST(frequency as FLOAT),
              CAST(azimuth as FLOAT),
              CAST(mdt + edt AS FLOAT) as downtilt,
              CAST('V2_Sinclair_SP42F2X_HF1P65LDF(D00)_831_ant' as VARCHAR)
          FROM
              {sites}
        ),
        terrain_elevations => CURSOR(
          SELECT
              CAST(x_4326 as DOUBLE),
              CAST(y_4326 as DOUBLE),
              CAST(dtm as DOUBLE),
              CAST(dsm as DOUBLE),
              CAST(attenuation as DOUBLE) as terrain_attenuation_dbm_per_meter
          FROM
              {clutter}
        ),
        antenna_patterns => CURSOR(
            SELECT
                CAST(antenna_name as VARCHAR),
                CAST(gain_str as DOUBLE),
                h_pattern_keys,
                h_pattern_values,
                v_pattern_keys,
                v_pattern_values
            FROM
              antenna_data
        ),
        rf_source_z_is_relative_to_terrain => true,
        geographic_coords => true,
        bin_dim_meters => 5.0,
        assumed_receiver_height_agl => 2.0 ,
        max_ray_travel_meters => 5000.0 ,
        initial_rays_per_source => 64 ,
        min_receiver_signal_strength_dbm => -110,
        default_source_height_agl_meters =>  30.0,
        ray_step_bin_multiple => 1.0,
        loop_grain_size => 8,
        num_rays_per_source => 1024)) where max_rf_signal_strength_dbm is not null """
    
    rf_output = pd.DataFrame(heavy.conn.execute(query))
    rf_output.columns = ['x','y','elevation_amsl_meters','rf_source_id','max_rf_signal_strength_dbm']
    load_table(rf_output[~rf_output['max_rf_signal_strength_dbm'].isna()], f'dev_full_aoi_detailed_rfp_area_{i}', schema = 'x Double, y Double, elevation_amsl_meters Double, rf_source_id TEXT, max_rf_signal_strength_dbm Double')
    return
    

In [22]:
#sites = "  antenna_data_raw_cell_index where long between  106.777  AND 106.79 AND lat between -6.24 AND -6.22  and frq_system = 'L1800' and ant_type NOT LIKE '%indoor%' limit 30"
import time 
import tqdm
import warnings
warnings.filterwarnings('ignore')
heavy = creat_connection()
for i in range(0,1):
    start = time.time()
    rf_output = execute_rfp(i,heavy)
    tqdm.tqdm.write(f"Base RFP processing time: {time.time()-start}")
    heavy.conn.execute(f'truncate table dev_full_aoi_baseline_rfp_area_{i}')
    load_table(rf_output[~rf_output['max_rf_signal_strength_dbm'].isna()], f'dev_full_aoi_baseline_rfp_area_{i}', schema = 'x Double, y Double, elevation_amsl_meters Double, rf_source_id TEXT, max_rf_signal_strength_dbm Double')
    tqdm.tqdm.write(f'Successfully loaded : dev_full_aoi_baseline_rfp_area_{i} to Heavy.')
    cell_list = rf_output['rf_source_id'].unique().tolist()
    try:
        heavy.conn.execute(f'truncate table dev_full_aoi_detailed_rfp_area_{i}')
    except:
        pass
    start_detailed = time.time()
    for cell in cell_list:
        try: 
            execute_rfp_detailed(i,cell,heavy)
           # tqdm.tqdm.write(f"processing time: {time.time()-start_detailed}")
        except Exception as e:
            Print("Error: ",e)
            continue
    tqdm.tqdm.write(f"Detailed processing time: {time.time()-start_detailed}")
    break


full_aoi_dsm_utm_area_0
Base RFP processing time: 25.21304440498352
Successfully loaded : dev_full_aoi_baseline_rfp_area_0 to Heavy.
Detailed processing time: 381.87028670310974


#### Post-processing

In [16]:
Detailed_RFP_Table = "dev_full_aoi_detailed_rfp_all_areas"
concate_sql = f"""create table {Detailed_RFP_Table} 
as 
select * from dev_full_aoi_detailed_rfp_area_0
union all 
select * from dev_full_aoi_detailed_rfp_area_1
union all 
select * from dev_full_aoi_detailed_rfp_area_2
union all 
select * from dev_full_aoi_detailed_rfp_area_3
union all 
select * from dev_full_aoi_detailed_rfp_area_4
union all 
select * from dev_full_aoi_detailed_rfp_area_5
union all 
select * from dev_full_aoi_detailed_rfp_area_6
union all 
select * from dev_full_aoi_detailed_rfp_area_7
union all 
select * from dev_full_aoi_detailed_rfp_area_8
union all 
select * from dev_full_aoi_detailed_rfp_area_9
union all 
select * from dev_full_aoi_detailed_rfp_area_10
union all 
select * from dev_full_aoi_detailed_rfp_area_11
union all 
select * from dev_full_aoi_detailed_rfp_area_12
union all 
select * from dev_full_aoi_detailed_rfp_area_13
union all 
select * from dev_full_aoi_detailed_rfp_area_14
union all 
select * from dev_full_aoi_detailed_rfp_area_15
union all 
select * from dev_full_aoi_detailed_rfp_area_16
union all 
select * from dev_full_aoi_detailed_rfp_area_17;"""


categorize_query = f"""create table {Detailed_RFP_Table}_categorized as select * from (
  with MaxValues As (
  select *, Max(max_rf_signal_strength_dbm) OVER (PARTITION BY cast(x as Text), cast(y as text))
  as max_dbm from {Detailed_RFP_Table}
)
select *, case when max_rf_signal_strength_dbm = max_dbm THEN 'Server'
when max_rf_signal_strength_dbm >= max_dbm -3 Then 'Neightbor'
ELSe 'Noise'
End as signal_category 
from MaxValues); """


#drop
nbrs_query = f"""
create table {Detailed_RFP_Table}_NBRs as
select
  cast(x as text) as x,
  cast(y as text) as y,
  elevation_amsl_meters,
  rf_source_id,
  max_rf_signal_strength_dbm
from
  {Detailed_RFP_Table}_categorized
where
  signal_category = 'Neightbor' ;"""
  
#drop
server_query = f"""create table {Detailed_RFP_Table}_servers as
select
  cast(x as text) as x,
  cast(y as text) as y,
  elevation_amsl_meters,
  rf_source_id,
  max_rf_signal_strength_dbm
from
  {Detailed_RFP_Table}_categorized
where
  signal_category = 'Server';"""

  
NRB_matrix_query = f"""create table {Detailed_RFP_Table}_NBR_Matrix as
select
  a.rf_source_id as source_cell,
  b.rf_source_id as neighbor_cell,
  count(*) as count_
from
  {Detailed_RFP_Table}_servers a
  inner join {Detailed_RFP_Table}_NBRs b on a.x = b.x
  and a.y = b.y
group by
  source_cell,
  neighbor_cell;"""


drop_1_query_1 = f" drop table {Detailed_RFP_Table}_servers;"
drop_1_query_2 = f" drop table {Detailed_RFP_Table}_NBRs;"

#drop
Noise_cal_query = f""" create table {Detailed_RFP_Table}_Noise as
        select
          cast(x as text) as x,
          cast(y as text) as y,
          elevation_amsl_meters,
          avg(max_rf_signal_strength_dbm) as noise,
          max_dbm,
          signal_category
        from
          {Detailed_RFP_Table}_categorized
        where
          signal_category = 'Noise'
        group by
          cast({Detailed_RFP_Table}_categorized.x as text),
          cast({Detailed_RFP_Table}_categorized.y as text),
          elevation_amsl_meters,
          max_dbm,
          signal_category ;"""
#drop
interf_cal_query =f"""create table  {Detailed_RFP_Table}_interf as
        select
          cast(x as text) as x,
          cast(y as text) as y,
          elevation_amsl_meters,
          avg(max_rf_signal_strength_dbm) as interference,
      count(*) as pilot_pollution,
          max_dbm,
          signal_category
        from
          {Detailed_RFP_Table}_categorized
        where
          signal_category = 'Neightbor'
        group by
          cast({Detailed_RFP_Table}_categorized.x as text),
          cast({Detailed_RFP_Table}_categorized.y as text),
          elevation_amsl_meters,
          max_dbm,
          signal_category ;"""
#drop 
server_cal_query = f"""create table {Detailed_RFP_Table}_server as
        select
          cast(x as text) as x,
          cast(y as text) as y,
          x as long,
          y as lat,
          elevation_amsl_meters,
          rf_source_id,
          avg(max_rf_signal_strength_dbm) as max_rf_signal_strength_dbm,
          max_dbm,
          signal_category
        from
          {Detailed_RFP_Table}_categorized
        where
          signal_category = 'Server'
        group by
          long,lat,
          cast({Detailed_RFP_Table}_categorized.x as text),
          cast({Detailed_RFP_Table}_categorized.y as text),
          elevation_amsl_meters,
          max_dbm,
          rf_source_id,
          signal_category; """
      
      
Master_tbl_query =f"""create table {Detailed_RFP_Table}_master as
        select
          s.x,
          s.y,
          s.long,
          s.lat,
          s.rf_source_id,
          s.max_rf_signal_strength_dbm,
          nb.interference,
          n.noise,
      nb.pilot_pollution
        from
          {Detailed_RFP_Table}_server s
          left join {Detailed_RFP_Table}_interf nb on cast(s.x as text) = cast(nb.x as text)
          and cast(s.y as text) = cast(nb.y as text)
          left join {Detailed_RFP_Table}_Noise n on s.x = n.x
          and s.y = n.y """
drop_2_query_1 = f" drop table {Detailed_RFP_Table}_server"
drop_2_query_2 = f" drop table {Detailed_RFP_Table}_interf"
drop_2_query_3 = f" drop table {Detailed_RFP_Table}_Noise"

sinr_query_1 = f"""alter table {Detailed_RFP_Table}_master add SINR double;"""
      
sinr_query_2 = f"""update {Detailed_RFP_Table}_master set interference = -140 where interference is null;"""
sinr_query_3 =f"""update {Detailed_RFP_Table}_master set pilot_pollution = 0 where pilot_pollution is null;"""
       
sinr_query_4 = f"""update {Detailed_RFP_Table}_master
      set SINR = 10*LOG10(POWER(10, max_rf_signal_strength_dbm/10)/(power(10,interference/10)+power(10,noise/10)))"""
     
cqi_query_1 = f"""alter table {Detailed_RFP_Table}_master add CQI int;"""
      
cqi_query_2 = f"""update {Detailed_RFP_Table}_master
      set CQI = CASE 
      when SINR <= -6.7 THEN 1
      when SINR >-6.7 and SINR <=-4.7 then 2
      when SINR >-4.7 and SINR <=-2.3 then 3
      when SINR >-2.3 and SINR <=0.2 then 4
      when SINR >0.2 and SINR <=2.4 then 5
      when SINR >2.4 and SINR <=4.3 then 6
      when SINR >4.3 and SINR <=5.9 then 7
      when SINR >5.9 and SINR <=8.1 then 8
      when SINR >8.1 and SINR <=10.3 then 9
      when SINR >10.3 and SINR <=11.7 then 10
      when SINR >11.7 and SINR <=14.1 then 11
      when SINR >14.1 and SINR <=16.3 then 12
      when SINR >16.3 and SINR <=18.7 then 13
      when SINR >18.7 and SINR <=21 then 14
      when SINR >21 then 15
      END;"""

sinr_db_query_1 = f"""alter table {Detailed_RFP_Table}_master add SINR_db float;"""
      
sinr_db_query_2 =f"""update {Detailed_RFP_Table}_master set SINR_db = (10*LN(SINR))/(LN(10))"""


post_process_nbr_query = f"""create table {Detailed_RFP_Table}_NBR_Matrix_temp as select * from (  With Rankedcounts As (
    select  
      source_cell ,
      neighbor_cell,
       count_,
    ROW_Number() over (Partition by source_cell order by count_ desc) as rank_
      from  {Detailed_RFP_Table}_NBR_Matrix ) 
      select source_cell,neighbor_cell, count_,rank_,
      case when rank_ = 1 then neighbor_cell end as'first_nbr',
       case when rank_ = 2 then neighbor_cell end as'second_nbr',
       case when rank_ = 3 then neighbor_cell end as'third_nbr' 
        from Rankedcounts )"""
nbr_matrex_2_query = f""" create table {Detailed_RFP_Table}_processed_NBRs as select a.source_cell, a.first_nbr ,b.second_nbr, c.third_nbr from 
   (select * from {Detailed_RFP_Table}_NBR_Matrix_temp where first_nbr is not null) a
   left join (select * {Detailed_RFP_Table}_NBR_Matrix_temp where second_nbr is not null) b
   on a.source_cell = b.source_cell
   left join (select * from {Detailed_RFP_Table}_NBR_Matrix_temp where third_nbr is not null) c
      on a.source_cell= c.source_cell  """

drop_3_1 = f"""drop table {Detailed_RFP_Table}_NBR_Matrix_temp """

query_list = [concate_sql,categorize_query ,nbrs_query ,server_query ,NRB_matrix_query ,drop_1_query_1 ,
              drop_1_query_2 ,Noise_cal_query,interf_cal_query ,server_cal_query,Master_tbl_query ,drop_2_query_1 ,
              drop_2_query_2 ,drop_2_query_3 ,sinr_query_1,sinr_query_2 ,sinr_query_3 ,sinr_query_4 ,cqi_query_1 ,
              cqi_query_2,sinr_db_query_1 ,sinr_db_query_2 , post_process_nbr_query,nbr_matrex_2_query,drop_3_1]

In [12]:
heavy = creat_connection()
start = time.time()
for i,query in enumerate(query_list):
    loop_start = time.time()
    heavy.conn.execute(query)
    print(f"Query: {i} executed in {time.time()-loop_start:.4f}. Total Time: {time.time()-start:.4f}.")


Query: 0 executed in 49.0685. Total Time: 49.0686.
Query: 1 executed in 154.1172. Total Time: 203.1859.
Query: 2 executed in 12.6385. Total Time: 215.8245.
Query: 3 executed in 30.0170. Total Time: 245.8416.
Query: 4 executed in 3.0915. Total Time: 248.9332.
Query: 5 executed in 0.0109. Total Time: 248.9442.
Query: 6 executed in 0.0099. Total Time: 248.9541.


ProgrammingError: SQL Error: From line 4, column 15 to line 4, column 30: Table 'dev_rfp_nbrs_3' not found

In [17]:
heavy = creat_connection()
start = time.time()
for i,query in enumerate(query_list[7:]):
    loop_start = time.time()
    heavy.conn.execute(query)
    print(f"Query: {i} executed in {time.time()-loop_start:.4f}. Total Time: {time.time()-start:.4f}.")


Query: 0 executed in 155.1069. Total Time: 155.1070.
Query: 1 executed in 32.6556. Total Time: 187.7628.
Query: 2 executed in 91.5560. Total Time: 279.3188.
Query: 3 executed in 10.0046. Total Time: 289.3234.
Query: 4 executed in 0.0109. Total Time: 289.3344.
Query: 5 executed in 0.0077. Total Time: 289.3421.
Query: 6 executed in 0.0097. Total Time: 289.3519.
Query: 7 executed in 0.2626. Total Time: 289.6144.
Query: 8 executed in 2.4002. Total Time: 292.0148.
Query: 9 executed in 2.3584. Total Time: 294.3733.
Query: 10 executed in 6.9051. Total Time: 301.2785.
Query: 11 executed in 0.2072. Total Time: 301.4858.
Query: 12 executed in 3.1550. Total Time: 304.6409.
Query: 13 executed in 0.1569. Total Time: 304.7978.
Query: 14 executed in 3.0595. Total Time: 307.8573.


## Delete Drafts

In [28]:
cell_list = rf_output['rf_source_id'].unique().tolist()
len(cell_list )

457

In [46]:
dfs = []
start = time.time()


for cell in cell_list:
    
    loop_start = time.time()
    print(cell)
    sites = f" full_aoi_all_site_data where cell_index = '{cell}'"

  #  df = load_data("full_aoi_all_site_data", f"where cell_index = '{cell}'")
  #  df.columns =  ['enodeb_function_name','cell_name','enodeb_id','cell_id','cell_index ',
 # 'site_id','height ','pow ',
 # 'frequency ','azimuth ','edt ','mdt ','antenna_type ','sector_id ','ant_type',
 # 'horizontal_beam' ,  'vertical_beam' ,
 # 'kabupaten' , 'ul_bw' , 'dl_bw', 'pow_w' , 'lat' , 'long' , 'x ',  'y ',
 # 'x_4326',  'y_4326', ' cell_suffex']
#['site_id','enodeb_function_name','cell_name ','enodeb_id ','cell_id','long','lat','site_type','antenna_type','ant_type',' mentum',' azimuth_ransys','azimuth',' edt',' mdt ','height ','frq_system ','rb_config ','ports ','mode ','pow','kecamatan',' kabupaten ','beam ','type_tower ','horizontal_beam ','vertical_beam ','gain ','ul_bw',' dl_bw ','sector_id', 'cell_index']  
   # clutter = f"""full_aoi_dsm_utm_area_0 where x_4326  between  {df['long'].iloc[0]}-0.03  AND {df['long'].iloc[0]}+0.03
   # AND y_4326 between {df['lat'].iloc[0]}-0.03  AND {df['lat'].iloc[0]}+0.03"""
    
    clutter = 'full_aoi_dsm_utm_area_0'
    query = f""" select * from TABLE(tf_rf_prop_max_signal(
        rf_sources => CURSOR(
          SELECT
              CAST(cell_index as VARCHAR),
              CAST(x_4326 as FLOAT),
              CAST(y_4326 as FLOAT),
              CAST(height as FLOAT) as adjusted_height,
              CAST((power(10,pow/100)*0.0000004) as FLOAT),
              CAST(frequency as FLOAT),
              CAST(azimuth as FLOAT),
              CAST(mdt + edt AS FLOAT) as downtilt,
              CAST('V2_Sinclair_SP42F2X_HF1P65LDF(D00)_831_ant' as VARCHAR)
          FROM
              {sites}
        ),
        terrain_elevations => CURSOR(
          SELECT
              CAST(x_4326 as DOUBLE),
              CAST(y_4326 as DOUBLE),
              CAST(dtm as DOUBLE),
              CAST(dsm as DOUBLE),
              CAST(attenuation as DOUBLE) as terrain_attenuation_dbm_per_meter
          FROM
              {clutter}
        ),
        antenna_patterns => CURSOR(
            SELECT
                CAST(antenna_name as VARCHAR),
                CAST(gain_str as DOUBLE),
                h_pattern_keys,
                h_pattern_values,
                v_pattern_keys,
                v_pattern_values
            FROM
              antenna_data
        ),
        rf_source_z_is_relative_to_terrain => true,
        geographic_coords => true,
        bin_dim_meters => 5.0,
        assumed_receiver_height_agl => 2.0 ,
        max_ray_travel_meters => 5000.0 ,
        initial_rays_per_source => 64 ,
        min_receiver_signal_strength_dbm => -110,
        default_source_height_agl_meters =>  30.0,
        ray_step_bin_multiple => 1.0,
        loop_grain_size => 8,
        num_rays_per_source => 1024)) where max_rf_signal_strength_dbm is not null """
   # heavy = creat_connection()
    print(f"Loop processing time (Create Connection): {time.time()-loop_start}")
    
    rfp_start = time.time()
    rf_output = pd.DataFrame(heavy.conn.execute(query))
    print(f"Loop processing time (RFP Execution): {time.time()-rfp_start}")
    rf_output.columns = ['x','y','elevation_amsl_meters','rf_source_id','max_rf_signal_strength_dbm']
  #  dfs.append(rf_output)
    print(f"Loop processing time: {time.time()-loop_start}")
print(f"Total processing time: {time.time()-start}")

DPK530_21
Loop processing time (Create Connection): 3.147125244140625e-05
Loop processing time (RFP Execution): 0.07790994644165039
Loop processing time: 0.07813763618469238
DPK740_21
Loop processing time (Create Connection): 6.9141387939453125e-06
Loop processing time (RFP Execution): 0.3547089099884033
Loop processing time: 0.35491275787353516
DPK461_21
Loop processing time (Create Connection): 7.3909759521484375e-06
Loop processing time (RFP Execution): 0.38268089294433594
Loop processing time: 0.382948637008667
DPK003_41
Loop processing time (Create Connection): 1.0728836059570312e-05
Loop processing time (RFP Execution): 0.3905479907989502
Loop processing time: 0.3907802104949951
DPK545_21
Loop processing time (Create Connection): 9.5367431640625e-06
Loop processing time (RFP Execution): 0.26552295684814453
Loop processing time: 0.2656979560852051
BOO153_21
Loop processing time (Create Connection): 1.9788742065429688e-05
Loop processing time (RFP Execution): 0.33536195755004883
Lo

KeyboardInterrupt: 

In [40]:
rf_output.rf_source_id.value_counts()

""


In [19]:
load_table(rf_output[~rf_output['max_rf_signal_strength_dbm'].isna()], 'dev_full_aoi_baseline_rfp_area_2', schema = 'x Double, y Double, elevation_amsl_meters Double, rf_source_id TEXT, max_rf_signal_strength_dbm Double')

/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [12]:
heavy = creat_connection()
df = pd.DataFrame(heavy.conn.execute(f"SELECT  distinct left(right(cell_name,4),2) as suffex, count(*) as countt from full_aoi_all_site_data group by suffex order by countt desc"
                                        ))
df.to_csv('cell_naming.csv',index=False)

In [16]:
df = load_data('antenna_data_raw_cell_index', 'where long between  106.777  AND 106.79 AND lat between -6.24 AND -6.22 limit 30')
df

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,JSX737,E_JSX737M41_Kebayoranlamaraya-TBG,E_JSX737TE1_Kebayoranlamaraya-TBG_TE03,432737,34,106.779831,-6.235931,MACRO,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,30,RT11.2m+POLE 12,65.0,8.0,16.5,20,20,3,JSX737_34
1,JSX758,E_JSX758M41_SommersetberlianBTSH-STP,E_JSX758HL1_SommersetberlianBTSH-STP_HL03,432758,31,106.780781,-6.223201,BTS HOTEL,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,30,GF - POLE 18-21M,65.0,8.0,16.5,20,20,3,JSX758_31
2,JKS567,E_JKS567M42_ItcpermatahjINDOOR,E_JKS567IE2_Itcpermatahj_IE02,133567,24,106.783661,-6.220677,IBC,INDOOR,INDOOR (INDOOR),...,KOTA ADMINISTRASI JAKARTA SELATAN,250,INDOOR,360.0,360.0,2.0,20,20,2,JKS567_24
3,JKS810,E_JKS810M41_Jalanlimo,E_JKS810EL1_Jalanlimoprovidenceparkemacro_EL01,133810,41,106.785051,-6.223901,EASY MACRO,AAU5940,"AAU5940 (Easy Macro, 1800/2100)",...,KOTA ADMINISTRASI JAKARTA SELATAN,25,SST42,70.0,NaN,NaN,20,20,1,JKS810_41
4,JKS567,E_JKS567M42_ItcpermatahjINDOOR,E_JKS567IF2_Itcpermatahj_IF01,133567,15,106.783661,-6.220677,IBC,INDOOR,INDOOR (INDOOR),...,KOTA ADMINISTRASI JAKARTA SELATAN,200,INDOOR,360.0,360.0,2.0,20,20,1,JKS567_15
5,JSX804,E_JSX804M41_JalanlimoBTSH-IFT,E_JSX804HR1_JalanlimoBTSH-IFT_HR02,432804,23,106.780871,-6.225841,BTS HOTEL,COMBA,"COMBA (8 Ports, 1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,55,GF - POLE 18-21M,65.0,9.3,13.6,20,20,2,JSX804_23
6,JKS567,E_JKS567M42_ItcpermatahjINDOOR,E_JKS567IE2_Itcpermatahj_IE01,133567,14,106.783661,-6.220677,IBC,INDOOR,INDOOR (INDOOR),...,KOTA ADMINISTRASI JAKARTA SELATAN,250,INDOOR,360.0,360.0,2.0,20,20,1,JKS567_14
7,JSX804,E_JSX804M41_JalanlimoBTSH-IFT,E_JSX804HL1_JalanlimoBTSH-IFT_HL03,432804,31,106.780871,-6.225841,BTS HOTEL,COMBA,"COMBA (8 Ports, 1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,30,GF - POLE 18-21M,65.0,9.3,13.6,20,20,3,JSX804_31
8,JSX758,E_JSX758M41_SommersetberlianBTSH-STP,E_JSX758HE1_SommersetberlianBTSH-STP_HE02,432758,24,106.780781,-6.223201,BTS HOTEL,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,45,GF - POLE 18-21M,65.0,8.0,16.5,20,20,2,JSX758_24
9,JSX758,E_JSX758M41_SommersetberlianBTSH-STP,E_JSX758HF1_SommersetberlianBTSH-STP_HF03,432758,35,106.780781,-6.223201,BTS HOTEL,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,50,GF - POLE 18-21M,65.0,8.0,16.5,20,20,3,JSX758_35


In [17]:
df.columns = ['site_id',' enodeb_function_name',' cell_name ','enodeb_id ','cell_id',' long','lat',' site_type',' antenna_type',' ant_type',' mentum',' azimuth_ransys','azimuth',' edt',' mdt ','height ','frq_system ','rb_config ','ports ','mode ','pow','kecamatan',' kabupaten ','beam ','type_tower ','horizontal_beam ','vertical_beam ','gain ','ul_bw',' dl_bw ','sector_id', 'cell_index']
df

,site_id,enodeb_function_name,cell_name,enodeb_id,cell_id,long,lat,site_type,antenna_type,ant_type,...,kabupaten,beam,type_tower,horizontal_beam,vertical_beam,gain,ul_bw,dl_bw,sector_id,cell_index
0,JSX737,E_JSX737M41_Kebayoranlamaraya-TBG,E_JSX737TE1_Kebayoranlamaraya-TBG_TE03,432737,34,106.779831,-6.235931,MACRO,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,30,RT11.2m+POLE 12,65.0,8.0,16.5,20,20,3,JSX737_34
1,JSX758,E_JSX758M41_SommersetberlianBTSH-STP,E_JSX758HL1_SommersetberlianBTSH-STP_HL03,432758,31,106.780781,-6.223201,BTS HOTEL,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,30,GF - POLE 18-21M,65.0,8.0,16.5,20,20,3,JSX758_31
2,JKS567,E_JKS567M42_ItcpermatahjINDOOR,E_JKS567IE2_Itcpermatahj_IE02,133567,24,106.783661,-6.220677,IBC,INDOOR,INDOOR (INDOOR),...,KOTA ADMINISTRASI JAKARTA SELATAN,250,INDOOR,360.0,360.0,2.0,20,20,2,JKS567_24
3,JKS810,E_JKS810M41_Jalanlimo,E_JKS810EL1_Jalanlimoprovidenceparkemacro_EL01,133810,41,106.785051,-6.223901,EASY MACRO,AAU5940,"AAU5940 (Easy Macro, 1800/2100)",...,KOTA ADMINISTRASI JAKARTA SELATAN,25,SST42,70.0,NaN,NaN,20,20,1,JKS810_41
4,JKS567,E_JKS567M42_ItcpermatahjINDOOR,E_JKS567IF2_Itcpermatahj_IF01,133567,15,106.783661,-6.220677,IBC,INDOOR,INDOOR (INDOOR),...,KOTA ADMINISTRASI JAKARTA SELATAN,200,INDOOR,360.0,360.0,2.0,20,20,1,JKS567_15
5,JSX804,E_JSX804M41_JalanlimoBTSH-IFT,E_JSX804HR1_JalanlimoBTSH-IFT_HR02,432804,23,106.780871,-6.225841,BTS HOTEL,COMBA,"COMBA (8 Ports, 1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,55,GF - POLE 18-21M,65.0,9.3,13.6,20,20,2,JSX804_23
6,JKS567,E_JKS567M42_ItcpermatahjINDOOR,E_JKS567IE2_Itcpermatahj_IE01,133567,14,106.783661,-6.220677,IBC,INDOOR,INDOOR (INDOOR),...,KOTA ADMINISTRASI JAKARTA SELATAN,250,INDOOR,360.0,360.0,2.0,20,20,1,JKS567_14
7,JSX804,E_JSX804M41_JalanlimoBTSH-IFT,E_JSX804HL1_JalanlimoBTSH-IFT_HL03,432804,31,106.780871,-6.225841,BTS HOTEL,COMBA,"COMBA (8 Ports, 1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,30,GF - POLE 18-21M,65.0,9.3,13.6,20,20,3,JSX804_31
8,JSX758,E_JSX758M41_SommersetberlianBTSH-STP,E_JSX758HE1_SommersetberlianBTSH-STP_HE02,432758,24,106.780781,-6.223201,BTS HOTEL,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,45,GF - POLE 18-21M,65.0,8.0,16.5,20,20,2,JSX758_24
9,JSX758,E_JSX758M41_SommersetberlianBTSH-STP,E_JSX758HF1_SommersetberlianBTSH-STP_HF03,432758,35,106.780781,-6.223201,BTS HOTEL,ASI4518R42v06,"ASI4518R42v06 (12 Ports, 900/1800/2100/2300)",...,KOTA ADMINISTRASI JAKARTA SELATAN,50,GF - POLE 18-21M,65.0,8.0,16.5,20,20,3,JSX758_35


In [ ]:
clutter = load_data('kota_jakarta_selatan_hr_utm_dsm', where = ' where x_4326  between  106.767  AND 106.8 AND y_4326 between -6.25 AND -6.21')
clutter.columns = ['x','y','AGL','dtm','x_4326','y_4326','clutter_type','clutter_id','attenuation','dsm']
clutter

In [5]:
rf_output['rf_source_id'].unique()

array(['JKS644_21', '14', 'JKS105_61', 'JKS644_31', 'JKS810_21',
       'JSX737_51', 'JSX737_21', 'JSX752_21', 'JKS733_21', 'JKS475_51',
       'JKS644_11', 'JSX737_41', 'JSX737_31', 'JKS475_31', 'JKS475_41',
       'JKS810_31', 'JKS475_11', 'JSX737_61', 'JSX737_11', 'JSX752_31',
       'JKS810_51', 'JKS810_41', 'JSX018_41'], dtype=object)

In [22]:
df['cell_index'].value_counts().sort_index()

cell_index
JKS567_14    1
JKS567_15    1
JKS567_24    1
JKS810_31    1
JKS810_41    1
JKS810_51    1
JSX737_14    1
JSX737_22    1
JSX737_34    1
JSX737_51    1
JSX758_11    1
JSX758_14    1
JSX758_15    1
JSX758_21    1
JSX758_24    1
JSX758_25    1
JSX758_31    1
JSX758_34    1
JSX758_35    1
JSX767_22    1
JSX767_32    1
JSX804_11    1
JSX804_12    1
JSX804_13    1
JSX804_21    1
JSX804_22    1
JSX804_23    1
JSX804_31    1
JSX804_32    1
JSX804_33    1
Name: count, dtype: int64

In [64]:
cell_list = rf_output['rf_source_id'].unique().tolist()
cell_list  

['COC577_21',
 'JKS758_41',
 'COC577_31',
 'JKS105_61',
 'JKS325_51',
 'JKS758_21',
 'JKS325_21',
 'JKS105_31',
 'JKS325_61',
 'JKS567_21',
 'JKS410_51',
 'JSX752_21',
 'JSX024_21',
 'JKS644_21',
 'JKS056_51',
 'JKS105_51',
 'JSX508_31',
 '14',
 'JSX737_51',
 'JKS105_21',
 'JKS644_31',
 'JKS733_21',
 'JSX024_31',
 'JSX737_21',
 'JKS810_21',
 'JKS057_61',
 'JSX508_21',
 'JSX767_21',
 'JKS410_21',
 'JKS357_21',
 'JSX758_21',
 'JSX154_21',
 'JKS056_31',
 'JKS357_11',
 'JKS475_21',
 'JKS056_21',
 'JSX024_11',
 'JKS475_31',
 'JSX737_41',
 'JKS644_41',
 'JKS325_31',
 'JKS410_41',
 'JKS644_11',
 'JSX804_21',
 'JKS810_51',
 'JKS325_11',
 'JSX737_31',
 'JKS056_61',
 'JKS325_71',
 'JKS733_31',
 'JKS758_51',
 'JKS105_41',
 'JKS683_11',
 'JKS057_21',
 'JSX508_11',
 'JKS357_31',
 'JKS105_11',
 'JKS410_31',
 'EPE077_81',
 'JSX737_61',
 'JKS567_31',
 'JSX018_21',
 'JSX737_11',
 'JKS810_41',
 'JSX804_11',
 'COC577_11',
 'EPE077_71',
 'JKS475_11',
 'JKS475_41',
 'JKS711_51',
 'JKS410_11',
 'JKS056_11',

In [65]:
dfs = []
start = time.time()

for cell in cell_list:
    print(cell)
    sites = f" antenna_data_raw_cell_index where cell_index = '{cell}'"
    clutter = 'kota_jakarta_selatan_hr_utm_dsm where x_4326  between  106.767  AND 106.8 AND y_4326 between -6.25 AND -6.21 '
    query = f""" select * from TABLE(tf_rf_prop_max_signal(
        rf_sources => CURSOR(
          SELECT
              CAST(cell_index as VARCHAR),
              CAST(long as FLOAT),
              CAST(lat as FLOAT),
              CAST(height as FLOAT) as adjusted_height,
              CAST((power(10,pow/10)*0.0001) as FLOAT),
              CAST(1800 as FLOAT),
              CAST(azimuth as FLOAT),
              CAST(mdt + edt AS FLOAT) as downtilt,
              CAST('V2_Sinclair_SP42F2X_HF1P65LDF(D00)_831_ant' as VARCHAR)
          FROM
              {sites}
        ),
        terrain_elevations => CURSOR(
          SELECT
              CAST(x_4326 as DOUBLE),
              CAST(y_4326 as DOUBLE),
              CAST(dtm as DOUBLE),
              CAST(dtm+AGL as DOUBLE),
              CAST(attenuation as DOUBLE) as terrain_attenuation_dbm_per_meter
          FROM
              {clutter}
        ),
        antenna_patterns => CURSOR(
            SELECT
                CAST(antenna_name as VARCHAR),
                CAST(gain_str as DOUBLE),
                h_pattern_keys,
                h_pattern_values,
                v_pattern_keys,
                v_pattern_values
            FROM
              antenna_data
        ),
        rf_source_z_is_relative_to_terrain => true,
        geographic_coords => true,
        bin_dim_meters => 5.0,
        assumed_receiver_height_agl => 2.0 ,
        max_ray_travel_meters => 5000.0 ,
        initial_rays_per_source => 64 ,
        min_receiver_signal_strength_dbm => -140,
        default_source_height_agl_meters =>  30.0,
        ray_step_bin_multiple => 1.0,
        loop_grain_size => 8,
        num_rays_per_source => 1024)) """
    
    heavy = creat_connection()
    rf_output = pd.DataFrame(heavy.conn.execute(query))
    rf_output.columns = ['x','y','elevation_amsl_meters','rf_source_id','max_rf_signal_strength_dbm']
    dfs.append(rf_output)
print(f"processing time: {time.time()-start}")


COC577_21
JKS758_41
COC577_31
JKS105_61
JKS325_51
JKS758_21
JKS325_21
JKS105_31
JKS325_61
JKS567_21
JKS410_51
JSX752_21
JSX024_21
JKS644_21
JKS056_51
JKS105_51
JSX508_31
14
JSX737_51
JKS105_21
JKS644_31
JKS733_21
JSX024_31
JSX737_21
JKS810_21
JKS057_61
JSX508_21
JSX767_21
JKS410_21
JKS357_21
JSX758_21
JSX154_21
JKS056_31
JKS357_11
JKS475_21
JKS056_21
JSX024_11
JKS475_31
JSX737_41
JKS644_41
JKS325_31
JKS410_41
JKS644_11
JSX804_21
JKS810_51
JKS325_11
JSX737_31
JKS056_61
JKS325_71
JKS733_31
JKS758_51
JKS105_41
JKS683_11
JKS057_21
JSX508_11
JKS357_31
JKS105_11
JKS410_31
EPE077_81
JSX737_61
JKS567_31
JSX018_21
JSX737_11
JKS810_41
JSX804_11
COC577_11
EPE077_71
JKS475_11
JKS475_41
JKS711_51
JKS410_11
JKS056_11
JSX752_31
JKS733_11
JSX752_11
JSX018_31
JKS056_41
JSX804_31
JKS810_31
JSX154_11
JKS057_31
JSX154_31
JSX018_11
JSX018_41
JSX758_31
JSX980_61
JKS567_11
JKS057_11
JKS057_41
JKS057_51
JKS810_11
JSX980_41
JKS057_71
JSX980_51
JSX767_11
JSX767_31
JSX758_11
JKS711_31
processing time: 1098.25075

In [50]:
dfs = []
cell_list = ['JKS567_21']
for cell in cell_list:
    print(cell)
    sites = f" antenna_data_raw_cell_index where cell_index = '{cell}'"
    clutter = 'kota_jakarta_selatan_hr_utm_dsm where x_4326  between  106.767  AND 106.8 AND y_4326 between -6.25 AND -6.21 '
    query = f""" select * from TABLE(tf_rf_prop_max_signal(
        rf_sources => CURSOR(
          SELECT
              CAST(cell_index as VARCHAR),
              CAST(long as FLOAT),
              CAST(lat as FLOAT),
              CAST(height as FLOAT) as adjusted_height,
              CAST((power(10,pow/10)*0.0001) as FLOAT),
              CAST(1800 as FLOAT),
              CAST(azimuth as FLOAT),
              CAST(mdt + edt AS FLOAT) as downtilt,
              CAST('GOLDEN_ANTENNA' as VARCHAR)
          FROM
              {sites}
        ),
        terrain_elevations => CURSOR(
          SELECT
              CAST(x_4326 as DOUBLE),
              CAST(y_4326 as DOUBLE),
              CAST(0 as DOUBLE),
              CAST(5 as DOUBLE),
              CAST(0.3 as DOUBLE) as terrain_attenuation_dbm_per_meter
          FROM
              {clutter}
        ),
        antenna_patterns => CURSOR(
            SELECT
                CAST(name as VARCHAR),
                CAST(gain as DOUBLE),
                h_pattern_keys,
                h_pattern_values,
                v_pattern_keys,
                v_pattern_values
            FROM
              new_antenna_data_manual_aoi_2
        ),
        rf_source_z_is_relative_to_terrain => true,
        geographic_coords => true,
        bin_dim_meters => 5.0,
        assumed_receiver_height_agl => 2.0 ,
        max_ray_travel_meters => 5000.0 ,
        initial_rays_per_source => 64 ,
        min_receiver_signal_strength_dbm => -140,
        default_source_height_agl_meters =>  30.0,
        ray_step_bin_multiple => 1.0,
        loop_grain_size => 8,
        num_rays_per_source => 1024)) """
    
    heavy = creat_connection()
    rf_output = pd.DataFrame(heavy.conn.execute(query))
    rf_output.columns = ['x','y','elevation_amsl_meters','rf_source_id','max_rf_signal_strength_dbm']
    dfs.append(rf_output)


JKS567_21


In [51]:
rf_output2 = pd.concat(dfs, ignore_index=True)
rf_output2

,x,y,elevation_amsl_meters,rf_source_id,max_rf_signal_strength_dbm
0,106.793747,-6.245976,5.0,14,NaN
1,106.793793,-6.245976,5.0,14,NaN
2,106.793839,-6.245976,5.0,14,NaN
3,106.793884,-6.245976,5.0,14,NaN
4,106.793839,-6.249663,5.0,JKS567_21,-138.167969
...,...,...,...,...,...
648076,106.799759,-6.210059,5.0,JKS567_21,-131.695023
648077,106.799805,-6.210059,5.0,JKS567_21,-131.671219
648078,106.799850,-6.210059,5.0,JKS567_21,-133.166931
648079,106.799896,-6.210059,5.0,JKS567_21,-134.662598


In [52]:
load_table(rf_output2[~rf_output2['max_rf_signal_strength_dbm'].isna()], 'dev_rfp_output_3_temp', schema = 'x Double, y Double, elevation_amsl_meters Double, rf_source_id TEXT, max_rf_signal_strength_dbm Double')

/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [12]:
rf_output2[~rf_output2['max_rf_signal_strength_dbm'].isna()]

,x,y,elevation_amsl_meters,rf_source_id,max_rf_signal_strength_dbm
0,106.775703,-6.248539,24.013245,JKS644_21,-135.596695
1,106.775749,-6.248539,24.013245,JKS644_21,-131.249207
2,106.775795,-6.248539,24.013245,JKS644_21,-124.696922
3,106.775841,-6.248539,24.013245,JKS644_21,-124.645699
6,106.775887,-6.248539,24.013245,JKS644_21,-124.622849
...,...,...,...,...,...
14905734,106.783348,-6.210958,-32746.535156,JSX018_41,-124.005989
14905737,106.783394,-6.210958,-32746.695312,JSX018_41,-126.230652
14905741,106.783440,-6.210958,-32746.857422,JSX018_41,-123.981026
14905743,106.783478,-6.210958,-32747.019531,JSX018_41,-135.231567
